In [ ]:
import sys
# ------------------------------------
sys.path.insert(1,'/Users/charlesh/Documents/Codes/')
sys.path.append('/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/METHODS')
sys.path.insert(0, '/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/COMPS')
sys.path.insert(0, '/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/METHODS/ATaCR/ATaCR_Python/OBStools')
sys.path.insert(0, '/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/METHODS/ATaCR/ATaCR_Python')

import math
import numpy as np
import librosa
from obspy import Trace
import librosa.display
import matplotlib.pyplot as plt
import matplotlib.image as img
import matplotlib
import sys
import obspy
import pickle as pkl
import glob as g
from obspy.clients.fdsn import Client
import datetime
import re
import math
from numpy import linalg as eigen
# import noisecut
import matplotlib.colors as mcolors
from scipy.stats import norm
import scipy.stats as stats
from scipy import fft
# import ntk
from cmcrameri import cm

import ObsQA
import ObsQA as ob
from ObsQA.plots import qtp
from ObsQA import *
import obstools as obs
import cmath
from comp_tools import *
import ObsQA.metrics as metrics
from pathlib import Path
from cmcrameri import cm
NoiseFolder = '/Users/charlesh/Documents/Codes/OBS_Methods/NOISE'
CompFolder = NoiseFolder + '/COMPS/ObsQA/figures'
MethodsFolder = NoiseFolder + '/METHODS/ATaCR'

plotfolder = NoiseFolder + '/COMPS/ObsQA/figures'
compfolder = CompFolder
ATaCR_Py_DataFolder = ob.io.dir_libraries(MethodsFolder)[1]
datafolder = ATaCR_Py_DataFolder['Py_DataParentFolder']
eventsfolder = ATaCR_Py_DataFolder['Py_CorrectedTraces']
catalog = pd.read_pickle(eventsfolder + '/event_catalog_updated.pkl')
catalog = pd.read_pickle('/Users/charlesh/Documents/Codes/OBS_Methods/NOISE/METHODS/ATaCR/ATaCR_Python/Metrics/EVENTS/EventMetrics_using_STA_avgTFs.pkl')

    # Concerning win_length:
    # Smaller values improve the temporal resolution of the STFT
    # (i.e. the ability to discriminate impulses that are closely spaced in time) at the expense
    # of frequency resolution (i.e. the ability to discriminate pure tones that are closely spaced
    # in frequency). This effect is known as the time-frequency localization trade-off and needs to
    # be adjusted according to the properties of the input signal

In [ ]:
sta = catalog.iloc[0]
evi = 0
d = ob.io.Get_ATaCR_CorrectedEvents(ATaCR_Py_DataFolder['Py_CorrectedTraces'],[sta.Events[evi]],sta.Network,sta.Station)
RawZ = d.Raw[0]
atacr_corrected = d.Corrected[0]

raw_z = RawZ['trZ'].copy()
atacr_z = atacr_corrected['ZP-21'].copy()
p = RawZ['trP'].copy()
fatacr,atacr_coh = PZCOH(p,atacr_z.copy())
hps_z, spectrograms = noisecut(raw_z.copy(), ret_spectrograms=True,win_length=200)
spectrograms_raw = spectrograms[0]
frequencies, times = spectrograms[3:]

hps_twice, spectrograms = noisecut(hps_z.copy(), ret_spectrograms=True,win_length=200)
spectrograms_hps = spectrograms[0]
frequencies, times = spectrograms[3:]
atacr_hps_z, spectrograms = noisecut(atacr_z.copy(), ret_spectrograms=True,win_length=200)
atacr_hps_z.stats.location = 'ATaCR > ' + atacr_hps_z.stats.location
spectrograms_atacr = spectrograms[0]
frequencies, times = spectrograms[3:]
_, spectrograms = noisecut(atacr_hps_z.copy(), ret_spectrograms=True,win_length=200)
spectrograms_atacr_hps = spectrograms[0]
frequencies, times = spectrograms[3:]

A_title = 'ATaCR Output Spectrogram'
B_title = 'HPS Output Spectrogram'
C_title = 'ATaCR>HPS Output Spectrogram'
plot_spectrograms(spectrograms_atacr, spectrograms_hps,spectrograms_atacr_hps, frequencies, times,A_title,B_title,C_title)
# plt.plot(hps_twice)

In [ ]:
# event_time,Network,stai = ('201203090709', '7D', 'M08A')
# # PreProcEvent,CorrectedEvent,ML_TFs = obs.io.GetML_EventData_and_TransferFunctions(event_time,Network,stai,ATaCR_ML_DataFolder['ML_PreProcEventData'],ATaCR_ML_DataFolder['ML_CorrectedTraces'],ATaCR_ML_DataFolder['ML_TransferFunctions'])
# # plt.figure(figsize=(15, 20))
# # print('------The standard ATaCR output------')
# # obs.plots.ML_fig_event_corrected(PreProcEvent,CorrectedEvent,evstream=None, TF_list=CorrectedEvent['TF_list'][0], fmin=1./150., fmax=2,prefix='Matlab Output ')
# PreProcEvent,CorrectedEvent,ML_TFs = obs.io.GetML_EventData_and_TransferFunctions(event_time,Network,stai,ATaCR_ML_DataFolder['ML_PreProcEventData'],ATaCR_ML_DataFolder['ML_CorrectedTraces'],ATaCR_ML_DataFolder['ML_TransferFunctions'])
# # ZObspy = PreProcEvent[PreProcEvent['channel']=='BHZ'].Trace.iloc[0]
# ZObspy = CorrectedEvent[CorrectedEvent['label']=='ZP-H'].Trace.iloc[0]
# hps_trace, spectrograms = noisecut(ZObspy.copy(), ret_spectrograms=True,resample_factor=1.0,win_length=200)
# # hps_trace, spectrograms = noisecut(ZObspy.copy(), ret_spectrograms=True,resample_factor=1.0,win_length=600)
# print('----NoiseCut Output----')
# plot_noisecut_spectrograms(*spectrograms,ymax=0.5)

In [ ]:
# fmin=0.01
# corrtf = 'ZP'
# print('---Plot and save event gather---')
# sta = catalog.iloc[0]
# ev = sta.Events[0]
# for n in range(len(catalog)):
#     sta = catalog.iloc[n]z
#     for i in range(len(sta.Events)):
#         d = obs.utilities.io.Get_ATaCR_CorrectedEvents(ATaCR_Py_DataFolder['Py_CorrectedTraces'],[sta.Events[i]],sta.Network,sta.Station)
#         # prefix = sta.Network + '.' + sta.Station + ' :: m' + str(sta.Magnitude_mw[i]) + ', z' + str(sta.Depth_KM[i]) + 'km, ' + sta.Events[i] + ' :: ' + corrtf + '-' + sta.Averaging[i].upper() + '.avg'
#         prefix = sta.Network + '.' + sta.Station + ' :: m' + str(sta.Magnitude_mw[i]) + ', z' + str(sta.Depth_KM[i]) + 'km, ' + sta.Events[i]
#         print('S:' + str(n+1) + '/' + str(len(catalog)) + ' E:' + str(i+1) + '/' + str(len(sta.Events)) + ' ' + prefix)
#         PreATaCR = d.iloc[0].RawZ
#         PreATaCR.stats.location = 'PreProc-ATaCR'
#         PostATaCR = d.iloc[0].Corrected[corrtf]
#         PostATaCR.stats.location = 'ATaCR-Corrected-' + PostATaCR.stats.location
#         NC_from_PreATaCR, spectrograms = noisecut(PreATaCR.copy(), ret_spectrograms=True,win_length=204)
#         NC_from_PostATaCR, spectrograms = noisecut(PostATaCR.copy(), ret_spectrograms=True,win_length=204)
#         before = [PreATaCR.copy(),PreATaCR.copy(),PostATaCR.copy()]
#         after = [PostATaCR.copy(),NC_from_PreATaCR.copy(),NC_from_PostATaCR.copy()]
#         # f = sta.Network + '.' + sta.Station + '.m' + str(sta.Magnitude_mw[i]) + '.' + sta.Events[i] + '.' + sta.Averaging[i].upper()
#         f = sta.Network + '.' + sta.Station + '.m' + str(sta.Magnitude_mw[i]) + '.' + sta.Events[i]
#         # obs.plots.plotPrePost(before,after,fmin=fmin,title=prefix,ysig=None)
#         plotPrePost(before,after,fmin=fmin,title=prefix,ysig=None)
#         plt.savefig ('/Users/charlesh/Documents/Codes/ATaCR/ObsQA/figures/updated_comps/' + f + '.png', dpi=100)

In [ ]:
# i = 0
# fmin = 0.01
# corrtf = 'ZP-21'
# n = np.where(catalog.Station=='LT20')[0][0]
# sta = catalog.iloc[n]
# d = obs.utilities.io.Get_ATaCR_CorrectedEvents(ATaCR_Py_DataFolder['Py_CorrectedTraces'],[sta.Events[i]],sta.Network,sta.Station)
# # prefix = sta.Network + '.' + sta.Station + ' :: m' + str(sta.Magnitude_mw[i]) + ', z' + str(sta.Depth_KM[i]) + 'km, ' + sta.Events[i] + ' :: ' + corrtf + '-' + sta.Averaging[i].upper() + '.avg'
# prefix = sta.Network + '.' + sta.Station + ' :: m' + str(sta.Magnitude_mw[i]) + ', z' + str(sta.Depth_KM[i]) + 'km, ' + sta.Events[i] + '  (' + str(fmin) + ' Hz highpass)'
# print('S:' + str(n+1) + '/' + str(len(catalog)) + ' E:' + str(i+1) + '/' + str(len(sta.Events)) + ' ' + prefix)
# # PreATaCR = d.iloc[0].RawZ.copy()
# PreATaCR = d.Obspy[0].trZ.copy()
# PreATaCR.stats.location = '(Raw) PreProc-ATaCR'
# PreATaCR.detrend()
# PostATaCR = d.iloc[0].Corrected[corrtf].copy()
# PostATaCR.stats.location = 'ATaCR-Corrected:' + PostATaCR.stats.location
# NC_from_PreATaCR, spectrograms = noisecut(PreATaCR.copy(), ret_spectrograms=True,win_length=204)
# NC_from_PreATaCR.stats.location = NC_from_PreATaCR.stats.location + '-Corrected'
# NC_from_PostATaCR, spectrograms = noisecut(PostATaCR.copy(), ret_spectrograms=True,win_length=204)
# NC_from_PostATaCR.stats.location = NC_from_PostATaCR.stats.location + '-Corrected'
# before = [PreATaCR.copy(),PreATaCR.copy(),PostATaCR.copy()]
# after = [PostATaCR.copy(),NC_from_PreATaCR.copy(),NC_from_PostATaCR.copy()]
# # f = sta.Network + '.' + sta.Station + '.m' + str(sta.Magnitude_mw[i]) + '.' + sta.Events[i] + '.' + sta.Averaging[i].upper()
# f = sta.Network + '.' + sta.Station + '.m' + str(sta.Magnitude_mw[i]) + '.' + sta.Events[i]
# # yl = np.max(np.abs(PostATaCR.copy().data))*0.5
# yl = None
# # xl=[0,4000]
# xl = None
# ev_llaz = [catalog[catalog.Station==sta.Station].Metadata.tolist()[0][-1][0].origins[0].latitude,catalog[catalog.Station==sta.Station].Metadata.tolist()[0][-1][0].origins[0].longitude,catalog[catalog.Station==sta.Station].Metadata.tolist()[0][-1][0].origins[0].depth/1000]
# ob.plots.plotPrePost(before,after,fmin=fmin,title=prefix,ysig=None,ev_llaz=ev_llaz,xl=xl)

In [ ]:
# # fmin=0.01
# # corrtf = 'ZP'
# # print('---Plot and save event gather---')
# # sta = catalog.iloc[0]
# # ev = sta.Events[0]
# # for n in range(len(catalog)):
# #     sta = catalog.iloc[n]
# #     for i in range(len(sta.Events)):
# #         d = obs.utilities.io.Get_ATaCR_CorrectedEvents(ATaCR_Py_DataFolder['Py_CorrectedTraces'],[sta.Events[i]],sta.Network,sta.Station)
# #         # prefix = sta.Network + '.' + sta.Station + ' :: m' + str(sta.Magnitude_mw[i]) + ', z' + str(sta.Depth_KM[i]) + 'km, ' + sta.Events[i] + ' :: ' + corrtf + '-' + sta.Averaging[i].upper() + '.avg'
# #         prefix = sta.Network + '.' + sta.Station + ' :: m' + str(sta.Magnitude_mw[i]) + ', z' + str(sta.Depth_KM[i]) + 'km, ' + sta.Events[i]
# #         print('S:' + str(n+1) + '/' + str(len(catalog)) + ' E:' + str(i+1) + '/' + str(len(sta.Events)) + ' ' + prefix)
# #         PreATaCR = d.iloc[0].RawZ
# #         PreATaCR.stats.location = 'PreProc-ATaCR'
# #         PostATaCR = d.iloc[0].Corrected[corrtf]
# #         PostATaCR.stats.location = 'ATaCR-Corrected-' + PostATaCR.stats.location
# #         NC_from_PreATaCR, spectrograms = noisecut(PreATaCR.copy(), ret_spectrograms=True,win_length=204)
# #         NC_from_PostATaCR, spectrograms = noisecut(PostATaCR.copy(), ret_spectrograms=True,win_length=204)
# #         before = [PreATaCR.copy(),PreATaCR.copy(),PostATaCR.copy()]
# #         after = [PostATaCR.copy(),NC_from_PreATaCR.copy(),NC_from_PostATaCR.copy()]
# #         # f = sta.Network + '.' + sta.Station + '.m' + str(sta.Magnitude_mw[i]) + '.' + sta.Events[i] + '.' + sta.Averaging[i].upper()
# #         f = sta.Network + '.' + sta.Station + '.m' + str(sta.Magnitude_mw[i]) + '.' + sta.Events[i]
# #         # obs.plots.plotPrePost(before,after,fmin=fmin,title=prefix,ysig=None)
# #         plotPrePost(before,after,fmin=fmin,title=prefix,ysig=None)
# #         plt.savefig ('/Users/charlesh/Documents/Codes/ATaCR/ObsQA/figures/comps/' + f + '.png', dpi=100)
# fmin = 0.01
# corrtf = 'ZP-21'
# ATaCR_Py_DataFolder = obs.utilities.io.Get_ATaCR_Dirs(MethodsFolder + '/ATaCR')
# eventsfolder = ATaCR_Py_DataFolder['Py_CorrectedTraces']
# catalog = pd.read_pickle(eventsfolder + '/event_catalog.pkl')
# for n in range(len(catalog)):
#     sta = catalog.iloc[n]
#     for i in range(len(sta.Events)):
#         try:
#             d = obs.utilities.io.Get_ATaCR_CorrectedEvents(ATaCR_Py_DataFolder['Py_CorrectedTraces'],[sta.Events[i]],sta.Network,sta.Station)
#             # prefix = sta.Network + '.' + sta.Station + ' :: m' + str(sta.Magnitude_mw[i]) + ', z' + str(sta.Depth_KM[i]) + 'km, ' + sta.Events[i] + ' :: ' + corrtf + '-' + sta.Averaging[i].upper() + '.avg'
#             prefix = sta.Network + '.' + sta.Station + ' :: m' + str(sta.Magnitude_mw[i]) + ', z' + str(sta.Depth_KM[i]) + 'km, ' + sta.Events[i] + '  (' + str(fmin) + ' Hz highpass)'
#             print('S:' + str(n+1) + '/' + str(len(catalog)) + ' E:' + str(i+1) + '/' + str(len(sta.Events)) + ' ' + prefix)
#             # PreATaCR = d.iloc[0].RawZ.copy()
#             PreATaCR = d.Obspy[0].trZ.copy()
#             PreATaCR.stats.location = '(Raw) PreProc-ATaCR'
#             PreATaCR.detrend()
#             PostATaCR = d.iloc[0].Corrected[corrtf].copy()
#             PostATaCR.stats.location = 'ATaCR-Corrected:' + PostATaCR.stats.location
#             NC_from_PreATaCR, spectrograms = noisecut(PreATaCR.copy(), ret_spectrograms=True,win_length=204)
#             NC_from_PreATaCR.stats.location = NC_from_PreATaCR.stats.location + '-Corrected'
#             NC_from_PostATaCR, spectrograms = noisecut(PostATaCR.copy(), ret_spectrograms=True,win_length=204)
#             NC_from_PostATaCR.stats.location = NC_from_PostATaCR.stats.location + '-Corrected'
#             before = [PreATaCR.copy(),PreATaCR.copy(),PostATaCR.copy()]
#             after = [PostATaCR.copy(),NC_from_PreATaCR.copy(),NC_from_PostATaCR.copy()]
#             # f = sta.Network + '.' + sta.Station + '.m' + str(sta.Magnitude_mw[i]) + '.' + sta.Events[i] + '.' + sta.Averaging[i].upper()
#             f = sta.Network + '.' + sta.Station + '.m' + str(sta.Magnitude_mw[i]) + '.' + sta.Events[i]
#             yl = np.max(np.abs(PostATaCR.copy().data))*0.5
#             # xl=[0,4000]
#             xl = None
#             ev_llaz = [catalog[catalog.Station==sta.Station].Metadata.tolist()[0][-1][0].origins[0].latitude,catalog[catalog.Station==sta.Station].Metadata.tolist()[0][-1][0].origins[0].longitude,catalog[catalog.Station==sta.Station].Metadata.tolist()[0][-1][0].origins[0].depth/1000]
#             ob.plots.plotPrePost(before,after,fmin=fmin,title=prefix,ysig=None,ev_llaz=ev_llaz)
#             # plt.savefig ('/Users/charlesh/Documents/Codes/ATaCR/ObsQA/figures/updated_comps/' + f + '.png', dpi=100)
#             plt.savefig(MethodsFolder + '/ObsQA/figures/SubArray34n/_ATaCR_CompactTraces/'+ f + '.png', dpi=100)
#             plt.close()
#         except:
#             continue

# # sta_llaz = [Zraw.stats.sac.stla,Zraw.stats.sac.stlo,np.abs(Zraw.stats.sac.stel)]
# # ev_llaz = [catalog[catalog.Station=='M08A'].Metadata.tolist()[0][-1][0].origins[0].latitude,catalog[catalog.Station=='M08A'].Metadata.tolist()[0][-1][0].origins[0].longitude,catalog[catalog.Station=='M08A'].Metadata.tolist()[0][-1][0].origins[0].depth/1000]
# # times = ob.io.get_arrivals(sta_llaz,ev_llaz)

In [ ]:
# fmin = 0.01
# corrtf = 'ZP-21'
# ATaCR_Py_DataFolder = obs.utilities.io.Get_ATaCR_Dirs(MethodsFolder + '/ATaCR')
# eventsfolder = ATaCR_Py_DataFolder['Py_CorrectedTraces']
# catalog = pd.read_pickle(eventsfolder + '/event_catalog.pkl')
# for n in range(len(catalog)):
#     sta = catalog.iloc[n]
#     for i in range(len(sta.Events)):
#         try:
#             d = obs.utilities.io.Get_ATaCR_CorrectedEvents(ATaCR_Py_DataFolder['Py_CorrectedTraces'],[sta.Events[i]],sta.Network,sta.Station)
#             print('S:' + str(n+1) + '/' + str(len(catalog)) + ' E:' + str(i+1) + '/' + str(len(sta.Events)))
#             ZObspy = d.Obspy[0].trZ.copy()
#             hps_trace, spectrograms = noisecut(ZObspy, ret_spectrograms=True,win_length=200)
#             plot_noisecut_spectrograms(*spectrograms,ymax=0.5,figsize=(24,10))
#             f = sta.Network + '.' + sta.Station + '.m' + str(sta.Magnitude_mw[i]) + '.' + sta.Events[i]
#             # plt.savefig ('/Users/charlesh/Documents/Codes/ATaCR/ObsQA/figures/noisecut_spectrograms/' + f + '_spec.png', dpi=100)
#             plt.savefig(MethodsFolder + '/ObsQA/figures/SubArray34n/_SupportFiles/HPS_Spec/'+ f + '_spec.png', dpi=100)
#             # before = [ZObspy]
#             # after = [hps_trace.copy()]
#             # ev_llaz = [catalog[catalog.Station==sta.Station].Metadata.tolist()[0][-1][0].origins[0].latitude,catalog[catalog.Station==sta.Station].Metadata.tolist()[0][-1][0].origins[0].longitude,catalog[catalog.Station==sta.Station].Metadata.tolist()[0][-1][0].origins[0].depth/1000]
#             # prefix = sta.Network + '.' + sta.Station + ' :: m' + str(sta.Magnitude_mw[i]) + ', z' + str(sta.Depth_KM[i]) + 'km, ' + sta.Events[i] + '  (' + str(fmin) + ' Hz highpass)'
#             # ob.plots.plotPrePost(before,after,fmin=fmin,title=prefix,ysig=None,ev_llaz=ev_llaz)
#             # plt.savefig ('/Users/charlesh/Documents/Codes/ATaCR/ObsQA/figures/noisecut_spectrograms/' + f + '_trace.png', dpi=100)
#             # plt.savefig('/Users/charlesh/Documents/Codes/ATaCR/ObsQA/figures/SubArray34n/_SupportFiles/HPS_Spec/'+ f + '.png', dpi=100)
#         except:
#             continue
# # # Trace inputs
# # Ain = ZObspy
# # Aout = hps_trace.copy()
# # # Bin = ZObspy.copy()
# # # Bout = CorrectedEvent[CorrectedEvent['label']=='ZP'].Trace.iloc[0].copy()
# # # Ain.stats.location = 'Input: ' + Ain.stats.location
# # # Bin.stats.location = 'Input: ' + Bin.stats.location
# # # Aout.stats.location = 'Output: ' + Aout.stats.location
# # # Bout.stats.location = 'Output: ' + Bout.stats.location
# # ob.plots.qtp([Ain,Aout],fmin=0.01)
